In [217]:
# Packages
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn import svm, tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier as DTC
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [218]:
# Loading
path = 'D:\\Lab\\RProjects\\ECON432\\425 data\\cleandata.csv'
data = pd.read_csv('cleandata.csv')

# Redefine symboling to three categories
def rangesymbol(newrange=[-1, 0, 1]):
    for i in range(data.shape[0]):
        if data['symboling'][i] <= -2:
            data['symboling'][i] = newrange[0]
        elif 2 > data['symboling'][i] > -2:
            data['symboling'][i] = newrange[1]
        elif data['symboling'][i] >= 2:
            data['symboling'][i] = newrange[2]


rangesymbol()
ind = ["wheelbase",'carlength','carheight','carwidth','curbweight','enginesize','boreratio',
                 'stroke','compressionratio','horsepower','peakrpm','citympg','highwaympg','price']
data.loc[:, ind] = (data.loc[:, ind] - data.loc[:, ind].min())/(data.loc[:, ind].max() - data.loc[:, ind].min())
data.describe()

,symboling,fueltype,aspiration,doornumber,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,...,highwaympg,price,hardtop,hatchback,sedan,wagon,fwd,rwd,ohc,other
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,...,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.273171,0.902439,0.180488,0.560976,0.985366,0.354419,0.491780,0.467317,0.493740,0.414106,...,0.388190,0.202540,0.039024,0.341463,0.468293,0.121951,0.585366,0.370732,0.858537,0.078049
std,0.478470,0.297446,0.385535,0.497483,0.120377,0.175562,0.184139,0.178767,0.203627,0.201971,...,0.181222,0.198323,0.194127,0.475361,0.500215,0.328031,0.493865,0.484183,0.349352,0.268905
min,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,1.000000,0.230321,0.376119,0.316667,0.350000,0.254849,...,0.236842,0.066283,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,0.303207,0.479104,0.433333,0.525000,0.359193,...,0.368421,0.128519,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,1.000000,1.000000,0.000000,1.000000,1.000000,0.460641,0.626866,0.550000,0.641667,0.561288,...,0.473684,0.282632,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [219]:
# Split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 2:data.shape[1]],
                                                    data['symboling'], test_size=0.33, random_state=317)


def repaccuracy(mod):
    print('Training score is:', mod.score(X_train, y_train), '\n',
      'Testing score is:', mod.score(X_test, y_test), '\n')
    
def confusionmatrix(predY, trueY):
    examp = np.hstack((predY, trueY))
    catt = list(set(trueY.iloc[:, 0]))
    catp = list(set(predY.iloc[:, 0]))
    catt.sort()
    catp.sort()
    # Just to make sure we have all categories:
    if len(catt) >= len(catp):
        catp = catt
    else:
        catt = catp
    confmat = np.zeros((len(catt), len(catp)))
    for i in range(len(catp)):
        for j in range(len(catt)):
            temp = examp == [catp[i], catt[j]]
            temp = temp.sum(axis=1)
            confmat[i, j] = sum(temp == 2)
    # In the confusion matrix, rows suggest predicted, columns suggest ground truth
    confmat = pd.DataFrame(confmat, index = catp, columns=catt)
    accuracy = sum([confmat.iat[i, i] for i in range(confmat.shape[0])])/len(examp)
    print('Accuracy is {}.'.format(accuracy))
    res = []
    res.append(accuracy)
    for i in range(len(catp)):
        print('Precision for yhat={} is {}.'.
              format(confmat.columns[i], confmat.iat[i, i]/sum([confmat.iat[j, i] for j in range(len(catt))])))
        res.append(confmat.iat[i, i]/sum([confmat.iat[j, i] for j in range(len(catt))]))
    for i in range(len(catt)):
        print('Recall for y={} is {}.'.
              format(confmat.columns[i], confmat.iat[i, i]/sum([confmat.iat[i, j] for j in range(len(catp))])))
        res.append(confmat.iat[i, i]/sum([confmat.iat[i, j] for j in range(len(catp))]))
    print(confmat, '\n')
    print('Result recorded', '\n')
    return res

## Logistic regression results

In [220]:
Res = []
#M1
# OVR,None penalty,newton-cg
logistic = LogisticRegression(multi_class="ovr",penalty = "none",solver = "newton-cg", max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.948905109489051 
 Testing score is: 0.6764705882352942 

Accuracy is 0.6764705882352942.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8409090909090909.
Precision for yhat=1 is 0.34782608695652173.
Recall for y=-1 is 0.3333333333333333.
Recall for y=0 is 0.7115384615384616.
Recall for y=1 is 0.6153846153846154.
     -1     0     1
-1  1.0   2.0   0.0
 0  0.0  37.0  15.0
 1  0.0   5.0   8.0 

Result recorded 



In [221]:
#M2
# OVR,None penalty,lbfgs
logistic = LogisticRegression(multi_class="ovr",penalty = "none",solver = "lbfgs",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.948905109489051 
 Testing score is: 0.6617647058823529 

Accuracy is 0.6617647058823529.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8181818181818182.
Precision for yhat=1 is 0.34782608695652173.
Recall for y=-1 is 0.3333333333333333.
Recall for y=0 is 0.7058823529411765.
Recall for y=1 is 0.5714285714285714.
     -1     0     1
-1  1.0   2.0   0.0
 0  0.0  36.0  15.0
 1  0.0   6.0   8.0 

Result recorded 



In [222]:
#M3
# OVR,None penalty,sag
logistic = LogisticRegression(multi_class="ovr",penalty = "none",solver = "sag",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.948905109489051 
 Testing score is: 0.7352941176470589 

Accuracy is 0.7352941176470589.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9090909090909091.
Precision for yhat=1 is 0.391304347826087.
Recall for y=-1 is 0.5.
Recall for y=0 is 0.7407407407407407.
Recall for y=1 is 0.75.
     -1     0     1
-1  1.0   1.0   0.0
 0  0.0  40.0  14.0
 1  0.0   3.0   9.0 

Result recorded 



In [223]:
#M4
# OVR,None penalty,saga
logistic = LogisticRegression(multi_class="ovr",penalty = "none",solver = "saga",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.9416058394160584 
 Testing score is: 0.7647058823529411 

Accuracy is 0.7647058823529411.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9318181818181818.
Precision for yhat=1 is 0.43478260869565216.
Recall for y=-1 is 0.5.
Recall for y=0 is 0.7592592592592593.
Recall for y=1 is 0.8333333333333334.
     -1     0     1
-1  1.0   1.0   0.0
 0  0.0  41.0  13.0
 1  0.0   2.0  10.0 

Result recorded 



In [224]:
#M5
# OVR,L1 ,liblinear
logistic = LogisticRegression(multi_class="ovr",penalty = "l1",solver = "liblinear",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8029197080291971 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [225]:
#M6
# OVR,L1 ,saga
logistic = LogisticRegression(multi_class="ovr",penalty = "l1",solver = "saga",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8029197080291971 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [226]:
#M7
# OVR,L2 ,liblinear
logistic = LogisticRegression(multi_class="ovr",penalty = "l2",solver = "liblinear",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8029197080291971 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [227]:
#M8
# OVR,L2 ,newton-cg
logistic = LogisticRegression(multi_class="ovr",penalty = "l2",solver = "newton-cg",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8029197080291971 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [228]:
#M9
# OVR,L2 ,lbfgs
logistic = LogisticRegression(multi_class="ovr",penalty = "l2",solver = "lbfgs",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8029197080291971 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [229]:
#M10
# OVR,L2 ,sag
logistic = LogisticRegression(multi_class="ovr",penalty = "l2",solver = "sag",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8029197080291971 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [230]:
#M11
# OVR,L2 ,saga,lamda = 1
logistic = LogisticRegression(multi_class="ovr",penalty = "l2",solver = "saga",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8029197080291971 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [231]:
#M12
# OVR,L2 ,saga,lambda = 1000
logistic = LogisticRegression(multi_class="ovr",penalty = "l2",solver = "saga",C = 0.001,max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.7226277372262774 
 Testing score is: 0.6470588235294118 

Accuracy is 0.6470588235294118.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 1.0.
Precision for yhat=1 is 0.0.
Recall for y=-1 is nan.
Recall for y=0 is 0.6470588235294118.
Recall for y=1 is nan.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  44.0  23.0
 1  0.0   0.0   0.0 

Result recorded 



In [232]:
#M13
# OVR,L2 ,saga lambda = 0.001
logistic = LogisticRegression(multi_class="ovr",penalty = "l2",solver = "saga",C = 1000,max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.9197080291970803 
 Testing score is: 0.7794117647058824 

Accuracy is 0.7794117647058824.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9545454545454546.
Precision for yhat=1 is 0.43478260869565216.
Recall for y=-1 is 1.0.
Recall for y=0 is 0.7636363636363637.
Recall for y=1 is 0.8333333333333334.
     -1     0     1
-1  1.0   0.0   0.0
 0  0.0  42.0  13.0
 1  0.0   2.0  10.0 

Result recorded 



In [233]:
#M14
# OVO,none,newton-cg
logistic = LogisticRegression(multi_class="multinomial",penalty = "none",solver = "newton-cg",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.9562043795620438 
 Testing score is: 0.6617647058823529 

Accuracy is 0.6617647058823529.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8409090909090909.
Precision for yhat=1 is 0.30434782608695654.
Recall for y=-1 is 0.25.
Recall for y=0 is 0.7115384615384616.
Recall for y=1 is 0.5833333333333334.
     -1     0     1
-1  1.0   2.0   1.0
 0  0.0  37.0  15.0
 1  0.0   5.0   7.0 

Result recorded 



In [234]:
#M15
# OVO,none,lbfgs
logistic = LogisticRegression(multi_class="multinomial",penalty = "none",solver = "lbfgs",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.9562043795620438 
 Testing score is: 0.6470588235294118 

Accuracy is 0.6470588235294118.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8181818181818182.
Precision for yhat=1 is 0.30434782608695654.
Recall for y=-1 is 0.2.
Recall for y=0 is 0.7058823529411765.
Recall for y=1 is 0.5833333333333334.
     -1     0     1
-1  1.0   3.0   1.0
 0  0.0  36.0  15.0
 1  0.0   5.0   7.0 

Result recorded 



In [235]:
#M16
#OVO,none,sag
logistic = LogisticRegression(multi_class="multinomial",penalty = "none",solver = "sag",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.9562043795620438 
 Testing score is: 0.7058823529411765 

Accuracy is 0.7058823529411765.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8636363636363636.
Precision for yhat=1 is 0.391304347826087.
Recall for y=-1 is 0.5.
Recall for y=0 is 0.7307692307692307.
Recall for y=1 is 0.6428571428571429.
     -1     0     1
-1  1.0   1.0   0.0
 0  0.0  38.0  14.0
 1  0.0   5.0   9.0 

Result recorded 



In [236]:
#M17
# OVO,none,saga
logistic = LogisticRegression(multi_class="multinomial",penalty = "none",solver = "saga",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.948905109489051 
 Testing score is: 0.7205882352941176 

Accuracy is 0.7205882352941176.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8863636363636364.
Precision for yhat=1 is 0.391304347826087.
Recall for y=-1 is 0.5.
Recall for y=0 is 0.7358490566037735.
Recall for y=1 is 0.6923076923076923.
     -1     0     1
-1  1.0   1.0   0.0
 0  0.0  39.0  14.0
 1  0.0   4.0   9.0 

Result recorded 



In [237]:
#M18
# OVO,l1,saga
logistic = LogisticRegression(multi_class="multinomial",penalty = "l1",solver = "saga",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8102189781021898 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [238]:
#M19
# OVO,l2,newton-cg
logistic = LogisticRegression(multi_class="multinomial",penalty = "l2",solver = "newton-cg",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8175182481751825 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [239]:
#M20
# OVO,l2,lbfgs
logistic = LogisticRegression(multi_class="multinomial",penalty = "l2",solver = "lbfgs",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8175182481751825 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [240]:
#M21
# OVO,l2,sag
logistic = LogisticRegression(multi_class="multinomial",penalty = "l2",solver = "sag",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8175182481751825 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [241]:
#M22
# OVO,l2,saga, lamda = 1
logistic = LogisticRegression(multi_class="multinomial",penalty = "l2",solver = "saga",max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8175182481751825 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [242]:
#M23
# OVO,l2,saga,lambda = 1000
logistic = LogisticRegression(multi_class="multinomial",penalty = "l2",solver = "saga",C =0.001,max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.7226277372262774 
 Testing score is: 0.6470588235294118 

Accuracy is 0.6470588235294118.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 1.0.
Precision for yhat=1 is 0.0.
Recall for y=-1 is nan.
Recall for y=0 is 0.6470588235294118.
Recall for y=1 is nan.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  44.0  23.0
 1  0.0   0.0   0.0 

Result recorded 



In [243]:
#M24
# OVO,l2,saga,lambda = 0.001
logistic = LogisticRegression(multi_class="multinomial",penalty = "l2",solver = "saga",C =1000,max_iter=10000)
lr = logistic.fit(X_train,y_train)
repaccuracy(lr)
Res.append(confusionmatrix(DataFrame(lr.predict(X_test)), DataFrame(y_test)))

Training score is: 0.9562043795620438 
 Testing score is: 0.75 

Accuracy is 0.75.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9318181818181818.
Precision for yhat=1 is 0.391304347826087.
Recall for y=-1 is 0.5.
Recall for y=0 is 0.7454545454545455.
Recall for y=1 is 0.8181818181818182.
     -1     0     1
-1  1.0   1.0   0.0
 0  0.0  41.0  14.0
 1  0.0   2.0   9.0 

Result recorded 



## KNN

In [244]:
def knn_score(n,weights,p):
    knn = KNeighborsClassifier(n_neighbors=n, weights = weights,p=p)
    knn = knn.fit(X_train,y_train)
    print(f"train score {knn.score(X_train,y_train)} for {n}")
    print(f"test score {knn.score(X_test,y_test)} for {n}")
    repaccuracy(knn)
    Res.append(confusionmatrix(DataFrame(knn.predict(X_test)), DataFrame(y_test)))

In [245]:
#M25
#n=1 uniform manhattan 
knn_score(1, "uniform",1)

train score 1.0 for 1
test score 0.8088235294117647 for 1
Training score is: 1.0 
 Testing score is: 0.8088235294117647 

Accuracy is 0.8088235294117647.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9090909090909091.
Precision for yhat=1 is 0.6086956521739131.
Recall for y=-1 is 0.3333333333333333.
Recall for y=0 is 0.8163265306122449.
Recall for y=1 is 0.875.
     -1     0     1
-1  1.0   2.0   0.0
 0  0.0  40.0   9.0
 1  0.0   2.0  14.0 

Result recorded 



In [246]:
#M26
# n=1 uniform euclidean
knn_score(1, "uniform",2)

train score 1.0 for 1
test score 0.7794117647058824 for 1
Training score is: 1.0 
 Testing score is: 0.7794117647058824 

Accuracy is 0.7794117647058824.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8636363636363636.
Precision for yhat=1 is 0.6086956521739131.
Recall for y=-1 is 0.25.
Recall for y=0 is 0.8260869565217391.
Recall for y=1 is 0.7777777777777778.
     -1     0     1
-1  1.0   2.0   1.0
 0  0.0  38.0   8.0
 1  0.0   4.0  14.0 

Result recorded 



In [247]:
#M27
# n=1 uniform minkowski
knn_score(1, "uniform", 3)

train score 1.0 for 1
test score 0.7794117647058824 for 1
Training score is: 1.0 
 Testing score is: 0.7794117647058824 

Accuracy is 0.7794117647058824.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8636363636363636.
Precision for yhat=1 is 0.6086956521739131.
Recall for y=-1 is 0.25.
Recall for y=0 is 0.8260869565217391.
Recall for y=1 is 0.7777777777777778.
     -1     0     1
-1  1.0   2.0   1.0
 0  0.0  38.0   8.0
 1  0.0   4.0  14.0 

Result recorded 



In [248]:
#M28
# n=5 uniform manhattan 
knn_score(5, "uniform",1)

train score 0.8467153284671532 for 5
test score 0.7941176470588235 for 5
Training score is: 0.8467153284671532 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9318181818181818.
Precision for yhat=1 is 0.5652173913043478.
Recall for y=-1 is nan.
Recall for y=0 is 0.7884615384615384.
Recall for y=1 is 0.8125.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  41.0  10.0
 1  0.0   3.0  13.0 

Result recorded 



In [249]:
#M29
# n=10 uniform manhattan 
knn_score(10, "uniform",1)

train score 0.7883211678832117 for 10
test score 0.8088235294117647 for 10
Training score is: 0.7883211678832117 
 Testing score is: 0.8088235294117647 

Accuracy is 0.8088235294117647.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.5217391304347826.
Recall for y=-1 is nan.
Recall for y=0 is 0.7818181818181819.
Recall for y=1 is 0.9230769230769231.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  11.0
 1  0.0   1.0  12.0 

Result recorded 



In [250]:
#M30
# n=1 distance manhattan
knn_score(1, "distance", 1)

train score 1.0 for 1
test score 0.8088235294117647 for 1
Training score is: 1.0 
 Testing score is: 0.8088235294117647 

Accuracy is 0.8088235294117647.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9090909090909091.
Precision for yhat=1 is 0.6086956521739131.
Recall for y=-1 is 0.3333333333333333.
Recall for y=0 is 0.8163265306122449.
Recall for y=1 is 0.875.
     -1     0     1
-1  1.0   2.0   0.0
 0  0.0  40.0   9.0
 1  0.0   2.0  14.0 

Result recorded 



In [251]:
#M31
# n=1 distance euclidean
knn_score(1, "distance", 2)

train score 1.0 for 1
test score 0.7794117647058824 for 1
Training score is: 1.0 
 Testing score is: 0.7794117647058824 

Accuracy is 0.7794117647058824.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8636363636363636.
Precision for yhat=1 is 0.6086956521739131.
Recall for y=-1 is 0.25.
Recall for y=0 is 0.8260869565217391.
Recall for y=1 is 0.7777777777777778.
     -1     0     1
-1  1.0   2.0   1.0
 0  0.0  38.0   8.0
 1  0.0   4.0  14.0 

Result recorded 



In [252]:
#M32
# n=1 distance minkowski
knn_score(1, "distance", 3)

train score 1.0 for 1
test score 0.7794117647058824 for 1
Training score is: 1.0 
 Testing score is: 0.7794117647058824 

Accuracy is 0.7794117647058824.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8636363636363636.
Precision for yhat=1 is 0.6086956521739131.
Recall for y=-1 is 0.25.
Recall for y=0 is 0.8260869565217391.
Recall for y=1 is 0.7777777777777778.
     -1     0     1
-1  1.0   2.0   1.0
 0  0.0  38.0   8.0
 1  0.0   4.0  14.0 

Result recorded 



In [253]:
#M33
# n=5 distance manhattan
knn_score(5, "distance", 1)

train score 1.0 for 5
test score 0.8676470588235294 for 5
Training score is: 1.0 
 Testing score is: 0.8676470588235294 

Accuracy is 0.8676470588235294.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.6521739130434783.
Recall for y=-1 is 1.0.
Recall for y=0 is 0.8431372549019608.
Recall for y=1 is 0.9375.
     -1     0     1
-1  1.0   0.0   0.0
 0  0.0  43.0   8.0
 1  0.0   1.0  15.0 

Result recorded 



In [254]:
#M34
# n=10 distance manhattan
knn_score(10, "distance", 1)

train score 1.0 for 10
test score 0.8529411764705882 for 10
Training score is: 1.0 
 Testing score is: 0.8529411764705882 

Accuracy is 0.8529411764705882.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9545454545454546.
Precision for yhat=1 is 0.6521739130434783.
Recall for y=-1 is 1.0.
Recall for y=0 is 0.84.
Recall for y=1 is 0.8823529411764706.
     -1     0     1
-1  1.0   0.0   0.0
 0  0.0  42.0   8.0
 1  0.0   2.0  15.0 

Result recorded 



## Neural Network


In [255]:
# we have 28 features, try 30 perceptrons
# M35
# relu adam constant 1 hidden layer
NN = MLPClassifier(max_iter = 2000,hidden_layer_sizes=(30,))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 0.9927007299270073 
 Testing score is: 0.8235294117647058 

Accuracy is 0.8235294117647058.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9545454545454546.
Precision for yhat=1 is 0.5652173913043478.
Recall for y=-1 is 0.5.
Recall for y=0 is 0.8076923076923077.
Recall for y=1 is 0.9285714285714286.
     -1     0     1
-1  1.0   1.0   0.0
 0  0.0  42.0  10.0
 1  0.0   1.0  13.0 

Result recorded 



In [256]:
# M36
# relu adam constant 2 hidden layer
NN = MLPClassifier(max_iter = 2000,hidden_layer_sizes=(30,30))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 1.0 
 Testing score is: 0.8235294117647058 

Accuracy is 0.8235294117647058.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9545454545454546.
Precision for yhat=1 is 0.5652173913043478.
Recall for y=-1 is 1.0.
Recall for y=0 is 0.8076923076923077.
Recall for y=1 is 0.8666666666666667.
     -1     0     1
-1  1.0   0.0   0.0
 0  0.0  42.0  10.0
 1  0.0   2.0  13.0 

Result recorded 



In [257]:
# M37
# relu adam constant 3 hidden layer
NN = MLPClassifier(max_iter = 2000,hidden_layer_sizes=(30,30,30))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 1.0 
 Testing score is: 0.8088235294117647 

Accuracy is 0.8088235294117647.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9318181818181818.
Precision for yhat=1 is 0.5652173913043478.
Recall for y=-1 is 0.5.
Recall for y=0 is 0.803921568627451.
Recall for y=1 is 0.8666666666666667.
     -1     0     1
-1  1.0   1.0   0.0
 0  0.0  41.0  10.0
 1  0.0   2.0  13.0 

Result recorded 



In [258]:
#M38
# relu lbfgs constant 2 hidden layer
NN = MLPClassifier(max_iter = 2000,solver = 'lbfgs',hidden_layer_sizes=(30,30))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 1.0 
 Testing score is: 0.8382352941176471 

Accuracy is 0.8382352941176471.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9545454545454546.
Precision for yhat=1 is 0.6086956521739131.
Recall for y=-1 is 0.3333333333333333.
Recall for y=0 is 0.84.
Recall for y=1 is 0.9333333333333333.
     -1     0     1
-1  1.0   1.0   1.0
 0  0.0  42.0   8.0
 1  0.0   1.0  14.0 

Result recorded 



In [259]:
# M39
# relu sgd constant 2 hidden layer
NN = MLPClassifier(max_iter = 20000,solver = 'sgd',hidden_layer_sizes=(30,30))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 0.7956204379562044 
 Testing score is: 0.7794117647058824 

Accuracy is 0.7794117647058824.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.43478260869565216.
Recall for y=-1 is nan.
Recall for y=0 is 0.7543859649122807.
Recall for y=1 is 0.9090909090909091.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  13.0
 1  0.0   1.0  10.0 

Result recorded 



In [260]:
# M40
# logistic adam constant 2 hidden layer
NN = MLPClassifier(max_iter = 20000,solver = 'adam',activation="logistic",hidden_layer_sizes=(30,30))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 0.9854014598540146 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9090909090909091.
Precision for yhat=1 is 0.6086956521739131.
Recall for y=-1 is nan.
Recall for y=0 is 0.8.
Recall for y=1 is 0.7777777777777778.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  40.0   9.0
 1  0.0   4.0  14.0 

Result recorded 



In [261]:
# M41
# logistic lbfgs constant 2 hidden layer
NN = MLPClassifier(max_iter = 20000,solver = 'lbfgs',activation="logistic",hidden_layer_sizes=(30,30))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 1.0 
 Testing score is: 0.8382352941176471 

Accuracy is 0.8382352941176471.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9318181818181818.
Precision for yhat=1 is 0.6521739130434783.
Recall for y=-1 is 0.3333333333333333.
Recall for y=0 is 0.8367346938775511.
Recall for y=1 is 0.9375.
     -1     0     1
-1  1.0   2.0   0.0
 0  0.0  41.0   8.0
 1  0.0   1.0  15.0 

Result recorded 



In [262]:
# M42
# logistic sgd constant 2 hidden layer
NN = MLPClassifier(max_iter = 20000,solver = 'sgd',activation="logistic",hidden_layer_sizes=(30,30))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 0.7226277372262774 
 Testing score is: 0.6470588235294118 

Accuracy is 0.6470588235294118.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 1.0.
Precision for yhat=1 is 0.0.
Recall for y=-1 is nan.
Recall for y=0 is 0.6470588235294118.
Recall for y=1 is nan.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  44.0  23.0
 1  0.0   0.0   0.0 

Result recorded 



In [263]:
# M43
# tanh adam constant 2 hidden layer
NN = MLPClassifier(max_iter = 20000,solver = 'adam',activation="tanh",hidden_layer_sizes=(30,30))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 1.0 
 Testing score is: 0.8235294117647058 

Accuracy is 0.8235294117647058.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9545454545454546.
Precision for yhat=1 is 0.5652173913043478.
Recall for y=-1 is 0.3333333333333333.
Recall for y=0 is 0.8235294117647058.
Recall for y=1 is 0.9285714285714286.
     -1     0     1
-1  1.0   1.0   1.0
 0  0.0  42.0   9.0
 1  0.0   1.0  13.0 

Result recorded 



In [264]:
# M44
# tanh lbfgs constant 2 hidden layer
NN = MLPClassifier(max_iter = 20000,solver = 'lbfgs',activation="tanh",hidden_layer_sizes=(30,30))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 1.0 
 Testing score is: 0.8676470588235294 

Accuracy is 0.8676470588235294.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9545454545454546.
Precision for yhat=1 is 0.6956521739130435.
Recall for y=-1 is 0.5.
Recall for y=0 is 0.8571428571428571.
Recall for y=1 is 0.9411764705882353.
     -1     0     1
-1  1.0   1.0   0.0
 0  0.0  42.0   7.0
 1  0.0   1.0  16.0 

Result recorded 



In [265]:
# M45
# tanh sgd constant 2 hidden layer
NN = MLPClassifier(max_iter = 20000,solver = 'sgd',activation="tanh",hidden_layer_sizes=(30,30))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 0.7883211678832117 
 Testing score is: 0.7352941176470589 

Accuracy is 0.7352941176470589.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.30434782608695654.
Recall for y=-1 is nan.
Recall for y=0 is 0.7166666666666667.
Recall for y=1 is 0.875.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  16.0
 1  0.0   1.0   7.0 

Result recorded 



In [266]:
# M46
# tanh lbfgs adaptive 2 hidden layer
NN = MLPClassifier(max_iter = 20000,solver = 'lbfgs',learning_rate='adaptive',activation="tanh",hidden_layer_sizes=(30,30))
NN = NN.fit(X_train,y_train)
repaccuracy(NN)
Res.append(confusionmatrix(DataFrame(NN.predict(X_test)), DataFrame(y_test)))

Training score is: 1.0 
 Testing score is: 0.8235294117647058 

Accuracy is 0.8235294117647058.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9545454545454546.
Precision for yhat=1 is 0.5652173913043478.
Recall for y=-1 is 0.5.
Recall for y=0 is 0.8076923076923077.
Recall for y=1 is 0.9285714285714286.
     -1     0     1
-1  1.0   1.0   0.0
 0  0.0  42.0  10.0
 1  0.0   1.0  13.0 

Result recorded 



## SVM

In [267]:
# M47
# OVO, linear
svm1 = svm.SVC(decision_function_shape='ovo', kernel='linear', max_iter=10000
               ).fit(X_train, y_train)
repaccuracy(svm1)
Res.append(confusionmatrix(DataFrame(svm1.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8248175182481752 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is 0.0.
Recall for y=0 is 0.7818181818181819.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   1.0
 0  1.0  43.0  11.0
 1  0.0   1.0  11.0 

Result recorded 



In [268]:
#M48
# OVO, LinearSVC
svmL = svm.LinearSVC(C=1, max_iter=2000000).fit(X_train, y_train)
repaccuracy(svmL)
Res.append(confusionmatrix(DataFrame(svmL.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8540145985401459 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [269]:
#M49
# OVO, LinearSVC, soft margin 0.2
svmL2 = svm.LinearSVC(C=0.2, max_iter=2000000).fit(X_train, y_train)
repaccuracy(svmL2)
Res.append(confusionmatrix(DataFrame(svmL2.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8102189781021898 
 Testing score is: 0.7941176470588235 

Accuracy is 0.7941176470588235.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.4782608695652174.
Recall for y=-1 is nan.
Recall for y=0 is 0.7678571428571429.
Recall for y=1 is 0.9166666666666666.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  12.0
 1  0.0   1.0  11.0 

Result recorded 



In [270]:
#M50
# OVO, polynomial, degree=2, coef0=5
svm2 = svm.SVC(C=1, decision_function_shape='ovo',
               kernel='poly', max_iter=10000000,
               degree=2, coef0=5, gamma='auto').fit(X_train, y_train)
repaccuracy(svm2)
Res.append(confusionmatrix(DataFrame(svm2.predict(X_test)), DataFrame(y_test)))


Training score is: 0.8248175182481752 
 Testing score is: 0.7794117647058824 

Accuracy is 0.7794117647058824.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.43478260869565216.
Recall for y=-1 is nan.
Recall for y=0 is 0.7543859649122807.
Recall for y=1 is 0.9090909090909091.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  13.0
 1  0.0   1.0  10.0 

Result recorded 



In [271]:
#M51
# OVO, polynomial, degree=3, coef0=5
svm2 = svm.SVC(C=1, decision_function_shape='ovo',
               kernel='poly', max_iter=10000000,
               degree=2, coef0=5, gamma='auto').fit(X_train, y_train)
repaccuracy(svm2)
Res.append(confusionmatrix(DataFrame(svm2.predict(X_test)), DataFrame(y_test)))


Training score is: 0.8248175182481752 
 Testing score is: 0.7794117647058824 

Accuracy is 0.7794117647058824.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.43478260869565216.
Recall for y=-1 is nan.
Recall for y=0 is 0.7543859649122807.
Recall for y=1 is 0.9090909090909091.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  13.0
 1  0.0   1.0  10.0 

Result recorded 



In [272]:
#M52
# OVO, polynomial, degree=5, coef0=5
svm2 = svm.SVC(C=1, decision_function_shape='ovo',
               kernel='poly', max_iter=10000000,
               degree=2, coef0=5, gamma='auto').fit(X_train, y_train)
repaccuracy(svm2)
Res.append(confusionmatrix(DataFrame(svm2.predict(X_test)), DataFrame(y_test)))


Training score is: 0.8248175182481752 
 Testing score is: 0.7794117647058824 

Accuracy is 0.7794117647058824.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.43478260869565216.
Recall for y=-1 is nan.
Recall for y=0 is 0.7543859649122807.
Recall for y=1 is 0.9090909090909091.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  13.0
 1  0.0   1.0  10.0 

Result recorded 



In [273]:
#M53
# OVO, polynomial, degree=2, coef0=1
svm2 = svm.SVC(C=1, decision_function_shape='ovo',
               kernel='poly', max_iter=10000000,
               degree=2, coef0=5, gamma='auto').fit(X_train, y_train)
repaccuracy(svm2)
Res.append(confusionmatrix(DataFrame(svm2.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8248175182481752 
 Testing score is: 0.7794117647058824 

Accuracy is 0.7794117647058824.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.43478260869565216.
Recall for y=-1 is nan.
Recall for y=0 is 0.7543859649122807.
Recall for y=1 is 0.9090909090909091.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  13.0
 1  0.0   1.0  10.0 

Result recorded 



In [274]:
#M54
# OVO, exponential, gamma=3
svm3 = svm.SVC(decision_function_shape='ovo',
               kernel='rbf', gamma=3, max_iter=100000).fit(X_train, y_train)
repaccuracy(svm3)
Res.append(confusionmatrix(DataFrame(svm3.predict(X_test)), DataFrame(y_test)))

Training score is: 0.9124087591240876 
 Testing score is: 0.75 

Accuracy is 0.75.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 1.0.
Precision for yhat=1 is 0.30434782608695654.
Recall for y=-1 is nan.
Recall for y=0 is 0.7213114754098361.
Recall for y=1 is 1.0.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  44.0  16.0
 1  0.0   0.0   7.0 

Result recorded 



In [275]:
#M55
# OVO, exponential, gamma=0.46
svm32 = svm.SVC(decision_function_shape='ovo',
               kernel='rbf', gamma=0.46, max_iter=100000).fit(X_train, y_train)
repaccuracy(svm32)
Res.append(confusionmatrix(DataFrame(svm32.predict(X_test)), DataFrame(y_test)))

Training score is: 0.8613138686131386 
 Testing score is: 0.7794117647058824 

Accuracy is 0.7794117647058824.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.43478260869565216.
Recall for y=-1 is nan.
Recall for y=0 is 0.7543859649122807.
Recall for y=1 is 0.9090909090909091.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  13.0
 1  0.0   1.0  10.0 

Result recorded 



In [276]:
#M56
# OVO, exponential, gamma=15
svm33 = svm.SVC(decision_function_shape='ovo',
               kernel='rbf', gamma=15, max_iter=100000).fit(X_train, y_train)
repaccuracy(svm33)
Res.append(confusionmatrix(DataFrame(svm33.predict(X_test)), DataFrame(y_test)))

Training score is: 0.9854014598540146 
 Testing score is: 0.7058823529411765 

Accuracy is 0.7058823529411765.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9772727272727273.
Precision for yhat=1 is 0.21739130434782608.
Recall for y=-1 is nan.
Recall for y=0 is 0.6935483870967742.
Recall for y=1 is 0.8333333333333334.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  43.0  18.0
 1  0.0   1.0   5.0 

Result recorded 



In [277]:
#M57
# OVO, sigmoid, gamma=auto
svm4 = svm.SVC(decision_function_shape='ovo',
               kernel='sigmoid', gamma='auto').fit(X_train, y_train)
repaccuracy(svm4)
Res.append(confusionmatrix(DataFrame(svm4.predict(X_test)), DataFrame(y_test)))

Training score is: 0.7226277372262774 
 Testing score is: 0.6470588235294118 

Accuracy is 0.6470588235294118.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 1.0.
Precision for yhat=1 is 0.0.
Recall for y=-1 is nan.
Recall for y=0 is 0.6470588235294118.
Recall for y=1 is nan.
     -1     0     1
-1  0.0   0.0   0.0
 0  1.0  44.0  23.0
 1  0.0   0.0   0.0 

Result recorded 



## Decision Trees

In [278]:
#M58
# gini, max depth 15
dtree = DTC(criterion='gini', max_depth=15, max_features=None).fit(X_train, y_train)
repaccuracy(dtree)
Res.append(confusionmatrix(DataFrame(dtree.predict(X_test)), DataFrame(y_test)))


Training score is: 1.0 
 Testing score is: 0.8382352941176471 

Accuracy is 0.8382352941176471.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9090909090909091.
Precision for yhat=1 is 0.6956521739130435.
Recall for y=-1 is 0.5.
Recall for y=0 is 0.851063829787234.
Recall for y=1 is 0.8421052631578947.
     -1     0     1
-1  1.0   1.0   0.0
 0  0.0  40.0   7.0
 1  0.0   3.0  16.0 

Result recorded 



In [279]:
#M59
# gini, max depth 50
dtree = DTC(criterion='gini', max_depth=50, max_features=None).fit(X_train, y_train)
repaccuracy(dtree)
Res.append(confusionmatrix(DataFrame(dtree.predict(X_test)), DataFrame(y_test)))

Training score is: 1.0 
 Testing score is: 0.8382352941176471 

Accuracy is 0.8382352941176471.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9090909090909091.
Precision for yhat=1 is 0.6956521739130435.
Recall for y=-1 is 1.0.
Recall for y=0 is 0.851063829787234.
Recall for y=1 is 0.8.
     -1     0     1
-1  1.0   0.0   0.0
 0  0.0  40.0   7.0
 1  0.0   4.0  16.0 

Result recorded 



In [280]:
#M60
# gini, max depth 15， max feature10
dtree = DTC(criterion='gini', max_depth=15, max_features=10).fit(X_train, y_train)
repaccuracy(dtree)
Res.append(confusionmatrix(DataFrame(dtree.predict(X_test)), DataFrame(y_test)))

Training score is: 1.0 
 Testing score is: 0.8676470588235294 

Accuracy is 0.8676470588235294.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9090909090909091.
Precision for yhat=1 is 0.782608695652174.
Recall for y=-1 is 0.3333333333333333.
Recall for y=0 is 0.8888888888888888.
Recall for y=1 is 0.9.
     -1     0     1
-1  1.0   2.0   0.0
 0  0.0  40.0   5.0
 1  0.0   2.0  18.0 

Result recorded 



In [281]:
#M61
# entropy, max depth 15
dtree = DTC(criterion='entropy', max_depth=15, max_features=None).fit(X_train, y_train)
repaccuracy(dtree)
Res.append(confusionmatrix(DataFrame(dtree.predict(X_test)), DataFrame(y_test)))


Training score is: 1.0 
 Testing score is: 0.8088235294117647 

Accuracy is 0.8088235294117647.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8409090909090909.
Precision for yhat=1 is 0.7391304347826086.
Recall for y=-1 is 0.3333333333333333.
Recall for y=0 is 0.8604651162790697.
Recall for y=1 is 0.7727272727272727.
     -1     0     1
-1  1.0   2.0   0.0
 0  0.0  37.0   6.0
 1  0.0   5.0  17.0 

Result recorded 



In [282]:
#M62
# entropy, max depth 50
dtree = DTC(criterion='entropy', max_depth=50, max_features=None).fit(X_train, y_train)
repaccuracy(dtree)
Res.append(confusionmatrix(DataFrame(dtree.predict(X_test)), DataFrame(y_test)))


Training score is: 1.0 
 Testing score is: 0.8529411764705882 

Accuracy is 0.8529411764705882.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9090909090909091.
Precision for yhat=1 is 0.7391304347826086.
Recall for y=-1 is 0.3333333333333333.
Recall for y=0 is 0.8695652173913043.
Recall for y=1 is 0.8947368421052632.
     -1     0     1
-1  1.0   2.0   0.0
 0  0.0  40.0   6.0
 1  0.0   2.0  17.0 

Result recorded 



In [283]:
#M63
# entropy, max depth 15, max feature 10
dtree = DTC(criterion='entropy', max_depth=15, max_features=10).fit(X_train, y_train)
repaccuracy(dtree)
Res.append(confusionmatrix(DataFrame(dtree.predict(X_test)), DataFrame(y_test)))


Training score is: 1.0 
 Testing score is: 0.9117647058823529 

Accuracy is 0.9117647058823529.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.9090909090909091.
Precision for yhat=1 is 0.9130434782608695.
Recall for y=-1 is 0.5.
Recall for y=0 is 0.9523809523809523.
Recall for y=1 is 0.875.
     -1     0     1
-1  1.0   1.0   0.0
 0  0.0  40.0   2.0
 1  0.0   3.0  21.0 

Result recorded 



In [284]:
#M64
# entropy, max depth 50, max feature 10
dtree = DTC(criterion='entropy', max_depth=50, max_features=10).fit(X_train, y_train)
repaccuracy(dtree)
Res.append(confusionmatrix(DataFrame(dtree.predict(X_test)), DataFrame(y_test)))


Training score is: 1.0 
 Testing score is: 0.8676470588235294 

Accuracy is 0.8676470588235294.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.8863636363636364.
Precision for yhat=1 is 0.8695652173913043.
Recall for y=-1 is 0.0.
Recall for y=0 is 0.9069767441860465.
Recall for y=1 is 0.8695652173913043.
     -1     0     1
-1  0.0   2.0   0.0
 0  1.0  39.0   3.0
 1  0.0   3.0  20.0 

Result recorded 



In [285]:
#M65
# entropy, max depth 50, max feature 15
dtree = DTC(criterion='entropy', max_depth=50, max_features=15).fit(X_train, y_train)
repaccuracy(dtree)
Res.append(confusionmatrix(DataFrame(dtree.predict(X_test)), DataFrame(y_test)))

Training score is: 1.0 
 Testing score is: 0.8235294117647058 

Accuracy is 0.8235294117647058.
Precision for yhat=-1 is 1.0.
Precision for yhat=0 is 0.8636363636363636.
Precision for yhat=1 is 0.7391304347826086.
Recall for y=-1 is 0.3333333333333333.
Recall for y=0 is 0.8636363636363636.
Recall for y=1 is 0.8095238095238095.
     -1     0     1
-1  1.0   2.0   0.0
 0  0.0  38.0   6.0
 1  0.0   4.0  17.0 

Result recorded 



In [286]:
#M66
# entropy, max depth 50, max feature 6
dtree = DTC(criterion='entropy', max_depth=50, max_features=6).fit(X_train, y_train)
repaccuracy(dtree)
Res.append(confusionmatrix(DataFrame(dtree.predict(X_test)), DataFrame(y_test)))

Training score is: 1.0 
 Testing score is: 0.8529411764705882 

Accuracy is 0.8529411764705882.
Precision for yhat=-1 is 0.0.
Precision for yhat=0 is 0.9090909090909091.
Precision for yhat=1 is 0.782608695652174.
Recall for y=-1 is 0.0.
Recall for y=0 is 0.8695652173913043.
Recall for y=1 is 0.8571428571428571.
     -1     0     1
-1  0.0   1.0   0.0
 0  1.0  40.0   5.0
 1  0.0   3.0  18.0 

Result recorded 



In [287]:
ResDF = pd.DataFrame(Res)
modellist = ['LR OVR,None penalty,newton-cg', 'OVR,None penalty,lbfgs', 'OVR,None penalty,sag', 
             'OVR,None penalty,saga', 'OVR,L1 ,liblinear', 'OVR,L1 ,saga', 
             'OVR,L2 ,liblinear', 'OVR,L2 ,newton-cg', 'OVR,L2 ,lbfgs', 
             'OVR,L2 ,sag', 'OVR,L2 ,saga,lambda=1', 'OVR,L2 ,saga,lambda=1000', 
             'OVR,L2 ,saga,lambda=0.001', 'OVO,none,newton-cg', 
             'OVO,none,lbfgs', 'OVO,none,sag', 'OVO,none,saga', 
             'OVO,l1,saga', 'OVO,l2,newton-cg', 'OVO,l2,lbfgs', 
             'OVO,l2,sag', 'OVO,l2,saga,lamda=1', 'OVO,L2 ,saga,lambda=1000', 
             'OVO,L2 ,saga,lambda=0.001', 'KNN,N=1,uniform,p=1', 
             'KNN,N=1,uniform,p=2', 'KNN,N=1,uniform,p=3', 'KNN,N=5,uniform,p=1', 
             'KNN,N=10,uniform,p=1', 'KNN,N=1,distance,p=1', 'KNN,N=1,distance,p=2', 
             'KNN,N=1,distance,p=3', 'KNN,N=5,distance,p=1', 'KNN,N=10,distance,p=1', 
             'MLP relu adam constant 1 hidden layer', 'relu adam constant 2 hidden layer', 'relu adam constant 3 hidden layer', 
             'relu lbfgs constant 2 hidden layer', 'relu sgd constant 2 hidden layer', 'logistic adam constant 2 hidden layer', 
             'logistic lbfgs constant 2 hidden layer', 'logistic sgd constant 2 hidden layer', 'tanh adam constant 2 hidden layer', 
             'tanh lbfgs constant 2 hidden layer', 'tanh sgd constant 2 hidden layer', 'tanh lbfgs adaptive 2 hidden layer', 
             'SVM OVO, linear', 'OVO, LinearSVC', 'OVO, LinearSVC, soft margin 0.2', 
             'OVO, polynomial, degree=2, coef0=5', 'OVO, polynomial, degree=3, coef0=5', 'OVO, polynomial, degree=5, coef0=5', 
             'OVO, polynomial, degree=2, coef0=1', 'OVO, exponential, gamma=3', 'OVO, exponential, gamma=0.46', 
             'OVO, exponential, gamma=15', 'OVO, sigmoid, gamma=auto', 'Decision Tree gini, max depth 15', 
             'gini, max depth 50', 'gini, max depth 15, max feature10', 'entropy, max depth 15', 
             'entropy, max depth 50', 'entropy, max depth 15, max feature 10', 'entropy, max depth 50, max feature 10', 
             'entropy, max depth 50, max feature 15', 'entropy, max depth 50, max feature 6']
ResDF.index = pd.Series(modellist)
ResDF.columns = ['Accuracy', 'Precision-1', 'Precision0', 'Precision+1', 
                 'Recall-1', 'Recall0', 'Recall+1']
ResDF.to_csv('mdres.csv', index=False, header=False)





